In [1]:
import pandas as pd
df = pd.read_csv('creditcard.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'creditcard.csv'

In [ ]:
#q1
x = df[df['Class']==1].count()
print(x)

In [ ]:
df['Time']

In [ ]:
Last_transaction = 172792.0/(60**2)
print(f"The last transaction occured {Last_transaction:.2f} hours later")

In [ ]:
for i in range(20):
    print(df['Time'][i])

In [ ]:
#q2
class_balance = 492/284807
print(class_balance)
#evaluation metric: AUC?  L2?

In [ ]:
from sklearn import preprocessing
#q3
#Time column: This dataset is European Credit cards over 48 hours.  Europe spans 3 time zones. divide time column by 3600
#and take mod 24, maybe +/- 2, especially since people can travel all over EU in a day.
#So not exactly normalization or standardization, but it is something that can help clustering
#Looking ahead, some will be optimization problems, so maybe normalization for the other features
#If I had more data, I might split it and weight the frauds by amount. But not here,
#because amount is a feature that may predict fraud
#Also, integer monetary amounts ($10 as opposed to $9.95+Tax) make me think that these particular transactions are 
#ATM withdrawals, but that is too much to assume

In [ ]:
df.Time= (df.Time/3600)%24

In [ ]:
df['Time']

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
df[[name  for name in list(df) if name not in ['Time','Class']]] = min_max_scaler.fit_transform(df[[name for name in list(df) if name not in ['Time','Class']]])

In [ ]:
df

In [ ]:
#q4
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df.drop('Class', axis=1)
y = df['Class']
X_r, X_s, y_r, y_s = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

In [ ]:
count = 0
for x in y_r:
    if x==1:
        count+=1
print(count)

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network  import MLPClassifier
from sklearn.ensemble  import RandomForestClassifier


In [ ]:
#no regularization
#I get an improvement in accuracy by doing the operation on the time column
classifiers = [SVC,DecisionTreeClassifier,MLPClassifier,RandomForestClassifier]
for classifier in classifiers:
    print(classifier," accuracy: ",accuracy_score(y_s,classifier().fit(X_r,y_r).predict(X_s)))

In [ ]:
from sklearn import tree
clf1 = tree.DecisionTreeClassifier(random_state=0, min_samples_split=10)
clf1.fit(X_r, y_r)
tree.plot_tree(clf1)

In [ ]:
print("unpruned accuracy: ",accuracy_score(y_s,clf1.predict(X_s)))

In [ ]:
#Tree pruning
Alpha=9.55 *(10**-5)
clf2 = DecisionTreeClassifier(random_state=0, ccp_alpha=Alpha)
clf2.fit(X_r, y_r)
tree.plot_tree(clf2)

In [ ]:
print("pruned accuracy: ",accuracy_score(y_s,clf2.predict(X_s)))

In [ ]:
#SVC regularization trial runs
C =[1,2,10,20,50]
kernel = ['linear','rbf']
for k in kernel:
    for c in C:
        svc = SVC(kernel = k,C=c).fit(X_r,y_r)
        print(f"accuracy with kernel={k:s} C={c:d}:",accuracy_score(y_s,svc.predict(X_s)))
        #increasing C slightly better accuracy for rbf

In [ ]:
#mlp regularization
alphas =[.001,.01,.1,.2,.5,1]
kernel = ['linear','rbf']
for alpha in alphas:
    mlp = MLPClassifier(alpha=alpha).fit(X_r,y_r)
    print(f"accuracy with penalty={alpha:.3f}:",accuracy_score(y_s,mlp.predict(X_s)))

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class PyTorchMLP(nn.Module):  # One hidden layer
    def __init__(self, n_hidden=10,n_hidden_2=10, epochs=100, eta=0.001, minibatch_size=50, seed=0):
        super(PyTorchMLP, self).__init__()
        self.random = np.random.RandomState(seed)  # shuffle mini batches
        self.n_hidden = n_hidden  # size of the hidden layer
        self.n_hidden_2 = n_hidden_2
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
        self.optimizer = None
        self.loss_func = torch.nn.CrossEntropyLoss()
        self.model = None

    def init_layers(self, _M:int, _K:int) -> None:
        # data structure
        self.model = torch.nn.Sequential(
            torch.nn.Linear(_M, self.n_hidden),
            torch.nn.Sigmoid(),
            torch.nn.Linear(self.n_hidden, self.n_hidden),
            torch.nn.Sigmoid(),
            torch.nn.Linear(self.n_hidden, self.n_hidden_2),
            torch.nn.Sigmoid(),
            torch.nn.Linear(self.n_hidden_2, self.n_hidden_2),
            torch.nn.Sigmoid(),
            torch.nn.Linear(self.n_hidden_2, _K),
        )
    
    def predict(self, _X):
        _X = torch.FloatTensor(_X)
        assert self.model is not None
        self.model.eval()
        with torch.no_grad():
            y_pred = np.argmax(self.model(_X), axis=1)
        self.model.train()
        return y_pred.numpy()

    def fit(self, _X_train, _y_train, info=False):
        import sys
        _X_train, _y_train = torch.FloatTensor(_X_train), torch.LongTensor(_y_train)
        n_features= _X_train.shape[1]
        n_output= np.unique(_y_train).shape[0]  # number of class labels
        
        self.init_layers(n_features, n_output)
        self.optimizer = torch.optim.Rprop(self.model.parameters(), lr=self.eta)  # connect model to optimizer

        for i in range(self.epochs):
            indices = np.arange(_X_train.shape[0])
            self.random.shuffle(indices)  # shuffle the data each epoch

            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                self.optimizer.zero_grad()
                
                net_out = self.model(_X_train[batch_idx])
                
                loss = self.loss_func(net_out, _y_train[batch_idx])
                loss.backward()
                self.optimizer.step()
                
                if info:
                    sys.stderr.write(f"\r{i+1:03d} Loss: {loss.item():6.5f}")
                    sys.stderr.flush()
        return self

In [ ]:
class MLP2(PyTorchMLP):
    def init_layers(self, _M, _K):
        self.model = torch.nn.Sequential(
            torch.nn.Linear(_M, self.n_hidden),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.n_hidden, self.n_hidden),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.n_hidden, self.n_hidden_2),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.n_hidden_2, self.n_hidden_2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.n_hidden_2, _K),
        )

In [ ]:
X_rs,y_rs,X_ts,y_ts = X_r.to_numpy(),y_r.to_numpy(),X_s.to_numpy(),y_s.to_numpy()

In [ ]:
nn1 = PyTorchMLP(n_hidden=20,n_hidden_2=25, epochs=100, eta=0.001, minibatch_size=X_rs.shape[0]).fit(X_rs,y_rs)
nn2 = MLP2(n_hidden=20,n_hidden_2=25, epochs=100, eta=0.001, minibatch_size=X_rs.shape[0]).fit(X_rs,y_rs)

In [ ]:
print(f'NN 1 Test Acc= {accuracy_score(nn1.predict(X_ts), y_ts):.5f}')
print(f'NN 2 (with dropout) Test Acc= {accuracy_score(nn2.predict(X_ts), y_ts):.5f}')
#A more robust model does not overfit but does not sacrifice accuracy, so it is preferred.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Alpha=9.55 *(10**-5)
rf1 = RandomForestClassifier()
rf2 = RandomForestClassifier(ccp_alpha=Alpha)

In [ ]:
scores = cross_val_score(rf1, X, y, cv=10)
print("random forest no pruning: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
scores = cross_val_score(rf2, X, y, cv=10)
print("random forest pruning: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=10)

In [ ]:
_X,_y = X.to_numpy(), y.to_numpy()

In [ ]:
_X,_y = X,y

In [ ]:
def tenfold(_clf,_X,_y):
    accuracies=[]
    for train_index, test_index in kf.split(_X,_y):
        _clf.fit(_X[train_index], _y[train_index])
        y_pred = _clf.predict(_X[test_index])
        accuracies += [accuracy_score(_y[test_index], y_pred)]
    return accuracies

In [ ]:
scores = tenfold(nn1, _X,_y)


In [ ]:
import numpy as np

In [ ]:
scores = np.array(scores)
print("neural net no dropout: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
scores = tenfold(nn2, _X,_y)
scores = np.array(scores)
print("neural net dropout: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))